## History
- Version 1 : num_inference_steps = 35  (about 33 seconds per image)
- Version 2 : num_inference_steps = 50  (about 41 seconds per image)

In [ ]:
!pip install --upgrade diffusers transformers -q

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm
from pathlib import Path
import torch
from diffusers import StableDiffusionPipeline
from transformers import pipeline, set_seed
from kaggle_secrets import UserSecretsClient

In [ ]:
df_train = pd.read_csv("/kaggle/input/stablediffusionv2-all-prompts/sd2_meta_211k_filter.csv")
#df_train = pd.read_csv("/kaggle/input/stablediffusionv2-all-prompts/sd2_meta_540k_part1.csv")
#df_train = pd.read_csv("/kaggle/input/stablediffusionv2-all-prompts/sd2_meta_540k_part2.csv")
#df_train = pd.read_csv("/kaggle/input/stablediffusionv2-all-prompts/sd2_meta_cc660k_part1.csv")
#df_train = pd.read_csv("/kaggle/input/stablediffusionv2-all-prompts/sd2_meta_cc660k_part2.csv")
#df_train = pd.read_csv("/kaggle/input/stablediffusionv2-all-prompts/sd2_meta_hard_filter.csv")
#df_train = pd.read_csv("/kaggle/input/stablediffusionv2-all-prompts/sd2_meta_laioncoco_p1.csv")
#df_train = pd.read_csv("/kaggle/input/stablediffusionv2-all-prompts/sd2_meta_pszemraj.csv")

In [ ]:
class CFG:
    device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
    seed = 42
    generator = torch.Generator(device).manual_seed(seed)
    image_gen_steps = 50 # 35
    image_gen_model_id = "stabilityai/stable-diffusion-2"
    image_gen_guidance_scale = 9
    
print(CFG.device)

In [ ]:
#user_secrets = UserSecretsClient()
#secret_hf_token = user_secrets.get_secret("secret_hf_token")

In [ ]:
image_gen_model = StableDiffusionPipeline.from_pretrained(
    CFG.image_gen_model_id, torch_dtype=torch.float16,
    revision="fp16", guidance_scale=9
)
image_gen_model = image_gen_model.to(CFG.device)

In [ ]:
def generate_image(prompt, model):
    image = model(
        prompt, num_inference_steps=CFG.image_gen_steps,
        generator=CFG.generator,
        guidance_scale=CFG.image_gen_guidance_scale
    ).images[0]
    return image

In [ ]:
!mkdir -p /kaggle/tmp/images/

In [ ]:
for row in df_train.itertuples(False):
    img = generate_image(row.prompt, image_gen_model)
    img = img.resize((512, 512))
    img.save(f'/kaggle/tmp/images/'+row.image_name)
    #break

In [ ]:
#!ls /kaggle/tmp/images/

In [ ]:
#!cp -r /kaggle/tmp/images/* ./

In [ ]:
#ls

In [ ]:
%%time
!tar -zcf images.tar.gz -C "/kaggle/tmp/images/" .